# Lab 03: Data Types and Efficient Formats

**Course:** Big Data

---

## 👤 Student Information

**Name:** `Your Name Here`

**Date:** `DD/MM/YYYY`

---

**Goal:** Master data type optimization and efficient storage formats to achieve significant memory and performance improvements.

## Learning Objectives

By the end of this lab, you will be able to:

1. **Optimize Data Types**: Reduce memory usage 5-10x through smart dtype selection
2. **Compare Storage Formats**: Understand trade-offs between CSV, Parquet, and Feather
3. **Configure Parquet**: Tune compression and row group settings
4. **Implement Partitioning**: Structure data for fast analytical queries

## Instructions

1. **Fill in your information above** before starting the lab
2. Read each cell carefully before running it
3. Implement the **TODO functions** when you see them
4. Run cells **from top to bottom** (Shift+Enter)
5. Check that output makes sense after each cell

---

## 📚 Libraries Used in This Lab

### Core Libraries

- **`pandas`** - DataFrame operations and I/O
- **`numpy`** - Random data generation
- **`pyarrow`** - Parquet and Arrow format support
- **`time`** - Performance measurement

### Why These Libraries?

- **PyArrow**: Industry-standard for columnar data, enables Parquet with advanced features
- **Pandas**: Our familiar DataFrame interface with dtype control

---

## 💡 The Cost of Bad Choices

**Real-world example**: A 100M row sales dataset

| Approach | Disk Size | RAM Usage | Load Time |
|----------|-----------|-----------|----------|
| Naive (object, CSV) | 50 GB | 80 GB | 15 min |
| Optimized (proper types, Parquet) | 2 GB | 5 GB | 20 sec |

**That's 25x smaller and 45x faster!**

---

## 1. Imports and Setup

In [ ]:
import json
import time
import os
from pathlib import Path

import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

print("✓ All imports successful!")
print(f"Pandas version: {pd.__version__}")
print(f"PyArrow version: {pa.__version__}")

## 2. Define Paths

In [ ]:
# Base directories
DATA_RAW = Path("../data/raw")
DATA_PROCESSED = Path("../data/processed")
RESULTS_DIR = Path("../results")

# File paths for this lab
ECOMMERCE_CSV = DATA_RAW / "ecommerce_5m.csv"
FORMATS_DIR = DATA_PROCESSED / "formats"
PARTITION_DIR = DATA_PROCESSED / "partitioned"
METRICS_PATH = RESULTS_DIR / "lab03_metrics.json"

# Ensure directories exist
DATA_RAW.mkdir(parents=True, exist_ok=True)
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)
FORMATS_DIR.mkdir(parents=True, exist_ok=True)
PARTITION_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print("Paths defined:")
print(f"  Source CSV: {ECOMMERCE_CSV}")
print(f"  Formats: {FORMATS_DIR}")
print(f"  Partitioned: {PARTITION_DIR}")
print(f"  Metrics: {METRICS_PATH}")

---

## 3. Dataset Generation

First, we generate a synthetic e-commerce dataset with 5 million rows.

**Columns:**
- `order_id`: Unique order identifier
- `product_id`: Product ID (1-50,000)
- `category`: Product category (15 unique values)
- `price`: Product price (0.01-999.99)
- `quantity`: Quantity ordered (1-100)
- `country`: Customer country (30 unique values)
- `timestamp`: Order timestamp

### TODO 1: `generate_ecommerce_data()`

Generate a synthetic e-commerce dataset.

**💡 Hints:**
- Use `np.random.seed(seed)` for reproducibility
- Use `np.random.randint()` for integer columns
- Use `np.random.choice()` for category/country columns
- Use `np.random.uniform()` for price
- Use `pd.date_range()` for timestamps

In [ ]:
def generate_ecommerce_data(path: Path, n_rows: int = 5_000_000, seed: int = 42) -> dict:
    """
    Generate a synthetic e-commerce dataset.
    
    Args:
        path: Where to save the CSV
        n_rows: Number of rows (default 5 million)
        seed: Random seed for reproducibility
    
    Returns:
        Dictionary with: {"rows": int, "cols": int, "size_mb": float}
    """
    # TODO: Implement this function
    # Step 1: Set random seed
    # Step 2: Define categories list (15 items)
    # Step 3: Define countries list (30 items)
    # Step 4: Generate each column
    # Step 5: Create DataFrame
    # Step 6: Save to CSV
    # Step 7: Return metadata
    pass

In [ ]:
# Generate the dataset (only if it doesn't exist)
if not ECOMMERCE_CSV.exists():
    print("Generating 5 million row e-commerce dataset...")
    print("(This may take 1-2 minutes)\n")
    
    start = time.perf_counter()
    metadata = generate_ecommerce_data(ECOMMERCE_CSV, n_rows=5_000_000)
    elapsed = time.perf_counter() - start
    
    print(f"Generated in {elapsed:.1f} seconds")
    print(f"Rows: {metadata['rows']:,}")
    print(f"Size: {metadata['size_mb']:.1f} MB")
else:
    size_mb = ECOMMERCE_CSV.stat().st_size / 1e6
    print(f"Dataset already exists: {size_mb:.1f} MB")

---

## Exercise 1: Data Type Optimization 🎯

### Part 1A: Baseline Measurement

First, let's see how much memory pandas uses with default dtypes.

In [ ]:
# Load with default dtypes
print("Loading CSV with default dtypes...")
start = time.perf_counter()
df_baseline = pd.read_csv(ECOMMERCE_CSV)
load_time_baseline = time.perf_counter() - start

print(f"Load time: {load_time_baseline:.2f} seconds")
print(f"\nDataFrame shape: {df_baseline.shape}")
print(f"\nColumn dtypes:")
print(df_baseline.dtypes)

In [ ]:
def measure_memory(df: pd.DataFrame) -> dict:
    """
    Measure memory usage of a DataFrame.
    
    Args:
        df: DataFrame to measure
    
    Returns:
        Dictionary with total and per-column memory in MB
    """
    # TODO: Implement this function
    # 1. Use df.memory_usage(deep=True) to get accurate memory
    # 2. Calculate total memory in MB
    # 3. Create dict with per-column info (dtype, memory, nunique)
    pass


# Measure baseline memory
baseline_memory = measure_memory(df_baseline)
print(f"\nTotal memory: {baseline_memory['total_mb']:.2f} MB")
print("\nPer-column breakdown:")
for col, info in baseline_memory['columns'].items():
    print(f"  {col}: {info['dtype']} - {info['memory_mb']:.2f} MB ({info['nunique']:,} unique)")

### Part 1B: Type Analysis

Let's analyze each column to determine the optimal type:

| Column | Current | Range/Values | Optimal Type |
|--------|---------|--------------|-------------|
| order_id | int64 | 0 to 5M | ? |
| product_id | int64 | 1 to 50,000 | ? |
| category | object | 15 unique | ? |
| price | float64 | 0.01 to 999.99 | ? |
| quantity | int64 | 1 to 100 | ? |
| country | object | 30 unique | ? |
| timestamp | object | dates | ? |

In [ ]:
# Analyze value ranges
print("Value ranges for numeric columns:")
for col in ['order_id', 'product_id', 'quantity']:
    print(f"  {col}: {df_baseline[col].min()} to {df_baseline[col].max()}")

print(f"\n  price: {df_baseline['price'].min():.2f} to {df_baseline['price'].max():.2f}")

print("\nUnique values for string columns:")
print(f"  category: {df_baseline['category'].nunique()} unique")
print(f"  country: {df_baseline['country'].nunique()} unique")

### TODO 3: Define Optimal Types

Based on your analysis, fill in the optimal types:

In [ ]:
def get_optimal_dtypes() -> dict:
    """
    Return the optimal dtypes for the ecommerce dataset.
    
    Returns:
        Dictionary mapping column names to dtype strings
    """
    # TODO: Fill in the optimal types based on your analysis
    return {
        'order_id': '???',      # 0 to 5M - which int type?
        'product_id': '???',    # 1 to 50000 - which int type?
        'category': '???',      # 15 unique strings
        'price': '???',         # 0.01 to 999.99
        'quantity': '???',      # 1 to 100 - which int type?
        'country': '???',       # 30 unique strings
    }


optimal_dtypes = get_optimal_dtypes()
print("Optimal dtypes:")
for col, dtype in optimal_dtypes.items():
    print(f"  {col}: {dtype}")

### Part 1C: Optimized Loading

Now let's reload with optimized types and measure the improvement.

In [ ]:
# Load with optimized dtypes
print("Loading CSV with optimized dtypes...")
start = time.perf_counter()
df_optimized = pd.read_csv(
    ECOMMERCE_CSV,
    dtype=optimal_dtypes,
    parse_dates=['timestamp']
)
load_time_optimized = time.perf_counter() - start

print(f"Load time: {load_time_optimized:.2f} seconds")
print(f"\nColumn dtypes:")
print(df_optimized.dtypes)

In [ ]:
# Measure optimized memory
optimized_memory = measure_memory(df_optimized)

print(f"Baseline memory: {baseline_memory['total_mb']:.2f} MB")
print(f"Optimized memory: {optimized_memory['total_mb']:.2f} MB")
print(f"\nReduction: {baseline_memory['total_mb'] / optimized_memory['total_mb']:.1f}x")

print("\nPer-column comparison:")
for col in baseline_memory['columns']:
    before = baseline_memory['columns'][col]['memory_mb']
    after = optimized_memory['columns'][col]['memory_mb']
    reduction = before / after if after > 0 else 0
    print(f"  {col}: {before:.1f} MB → {after:.1f} MB ({reduction:.1f}x)")

### Part 1D: Speed Impact

Smaller types aren't just about memory — they're also faster!

In [ ]:
# Benchmark groupby operation
print("Benchmarking groupby operation...\n")

# Baseline
start = time.perf_counter()
_ = df_baseline.groupby('category')['price'].sum()
groupby_baseline = time.perf_counter() - start

# Optimized
start = time.perf_counter()
_ = df_optimized.groupby('category')['price'].sum()
groupby_optimized = time.perf_counter() - start

print(f"Groupby baseline: {groupby_baseline:.4f} sec")
print(f"Groupby optimized: {groupby_optimized:.4f} sec")
print(f"Speedup: {groupby_baseline / groupby_optimized:.2f}x")

In [ ]:
# Benchmark filter operation
print("Benchmarking filter operation...\n")

# Baseline
start = time.perf_counter()
_ = df_baseline[df_baseline['country'] == 'Spain']
filter_baseline = time.perf_counter() - start

# Optimized
start = time.perf_counter()
_ = df_optimized[df_optimized['country'] == 'Spain']
filter_optimized = time.perf_counter() - start

print(f"Filter baseline: {filter_baseline:.4f} sec")
print(f"Filter optimized: {filter_optimized:.4f} sec")
print(f"Speedup: {filter_baseline / filter_optimized:.2f}x")

### 💡 Key Insight: Category dtype

The `category` dtype is especially powerful:

```python
# Internally stored as:
# Dictionary: {0: 'Electronics', 1: 'Clothing', ...}
# Codes: [0, 1, 0, 2, 1, ...]  (integers!)
```

Benefits:
- Groupby operates on integers, not strings
- Comparisons use integer codes
- Memory scales with unique values, not row count

---

## Exercise 2: Format Comparison 📊

Now let's compare different storage formats.

### Part 2A & 2B: Write and Measure Formats

In [ ]:
def benchmark_formats(df: pd.DataFrame, base_path: Path) -> dict:
    """
    Benchmark different file formats for writing and reading.
    
    Args:
        df: DataFrame to benchmark
        base_path: Directory to save files
    
    Returns:
        Dictionary with timing and size results
    """
    # TODO: Implement this function
    # For each format (CSV, CSV.gz, Parquet variants, Feather):
    # 1. Time the write operation
    # 2. Record file size
    # 3. Time full read
    # 4. Time partial read (3 columns) where supported
    pass


# Run benchmarks
print("Benchmarking file formats...")
print("(This may take a few minutes)\n")

format_results = benchmark_formats(df_optimized, FORMATS_DIR)

# Display results
print("\nResults:")
print("-" * 70)
print(f"{'Format':<20} {'Size (MB)':<12} {'Write (s)':<12} {'Read (s)':<12} {'3-col (s)':<12}")
print("-" * 70)
for fmt, data in format_results.items():
    partial = f"{data['read_partial_sec']:.3f}" if data['read_partial_sec'] else "N/A"
    print(f"{fmt:<20} {data['size_mb']:<12.1f} {data['write_sec']:<12.3f} {data['read_full_sec']:<12.3f} {partial:<12}")

### 💡 Key Insights: Formats

| Format | Best For |
|--------|----------|
| CSV | Universal exchange, debugging, small files |
| Parquet (zstd) | Long-term storage, analytics |
| Feather | Fast Python/R exchange, caching |

**Column Selection**: Only Parquet and Feather can read specific columns without loading the entire file!

---

## Exercise 3: Parquet Deep Dive 🔬

### Part 3A: Inspect Metadata

In [ ]:
def inspect_parquet(path: Path) -> dict:
    """
    Inspect Parquet file metadata.
    
    Args:
        path: Path to Parquet file
    
    Returns:
        Dictionary with metadata information
    """
    # TODO: Implement this function
    # 1. Open with pq.ParquetFile(path)
    # 2. Get num_row_groups, schema, etc.
    # 3. For each row group, get statistics
    pass


# Inspect the Parquet file
pq_path = FORMATS_DIR / 'data_zstd.parquet'
if pq_path.exists():
    pq_info = inspect_parquet(pq_path)
    print(f"Row Groups: {pq_info['num_row_groups']}")
    print(f"Total Rows: {pq_info['num_rows']:,}")
    print(f"\nSchema:\n{pq_info['schema']}")
else:
    print("Run the format benchmark first to generate the Parquet file.")

### Part 3B: Row Group Size Experiment

In [ ]:
def benchmark_row_group_sizes(df: pd.DataFrame, base_path: Path, 
                              sizes: list = [10_000, 100_000, 1_000_000]) -> dict:
    """
    Benchmark different row group sizes.
    
    Args:
        df: DataFrame to benchmark
        base_path: Directory to save files
        sizes: List of row group sizes to test
    
    Returns:
        Dictionary with results for each size
    """
    # TODO: Implement this function
    # For each row group size:
    # 1. Write with df.to_parquet(path, row_group_size=size)
    # 2. Count row groups with pq.ParquetFile
    # 3. Time read operation
    pass


# Run benchmark
print("Benchmarking row group sizes...\n")
rg_results = benchmark_row_group_sizes(df_optimized, FORMATS_DIR)

print("Results:")
print("-" * 60)
print(f"{'Row Group Size':<20} {'# Groups':<12} {'Size (MB)':<12} {'Read (s)':<12}")
print("-" * 60)
for size, data in rg_results.items():
    print(f"{size:<20,} {data['num_row_groups']:<12} {data['file_size_mb']:<12.1f} {data['read_sec']:<12.3f}")

### Part 3C: Predicate Pushdown

In [ ]:
def benchmark_predicate_pushdown(path: Path, column: str, threshold: float) -> dict:
    """
    Benchmark reading with and without predicate pushdown.
    
    Args:
        path: Path to Parquet file
        column: Column to filter on
        threshold: Value threshold for filter
    
    Returns:
        Dictionary with timing results
    """
    # TODO: Implement this function
    # 1. Read without filter, time it
    # 2. Read with filters=[(column, '>', threshold)], time it
    # 3. Calculate speedup
    pass


# Run predicate pushdown benchmark
pq_path = FORMATS_DIR / 'data_zstd.parquet'
if pq_path.exists():
    print("Benchmarking predicate pushdown...\n")
    pushdown_results = benchmark_predicate_pushdown(pq_path, 'price', 500.0)
    
    print(f"Without filter: {pushdown_results['no_filter_sec']:.3f} sec ({pushdown_results['no_filter_rows']:,} rows)")
    print(f"With filter: {pushdown_results['with_filter_sec']:.3f} sec ({pushdown_results['with_filter_rows']:,} rows)")
    print(f"\nSpeedup: {pushdown_results['speedup']:.2f}x")
else:
    print("Run the format benchmark first.")

### 💡 Key Insight: Predicate Pushdown

Parquet stores min/max statistics for each column chunk. When you filter:

1. Parquet checks statistics first
2. Skips entire row groups that can't match
3. Only reads relevant data

This is why analytical queries on Parquet are so fast!

---

## Exercise 4: Partitioning Strategies 📂

### Part 4A: Add Partition Columns

In [ ]:
def add_partition_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add year, month, day columns from timestamp for partitioning.
    
    Args:
        df: DataFrame with 'timestamp' column
    
    Returns:
        DataFrame with added partition columns
    """
    # TODO: Implement this function
    # 1. Convert timestamp to datetime if needed
    # 2. Extract year, month, day
    # 3. Use smallest int types (uint16, uint8)
    pass


# Add partition columns
df_partitioned = add_partition_columns(df_optimized)
print("Added partition columns:")
print(df_partitioned[['timestamp', 'year', 'month', 'day']].head())

### Part 4B: Implement Partitioning Strategies

In [ ]:
def benchmark_partitioning(df: pd.DataFrame, base_path: Path) -> dict:
    """
    Benchmark different partitioning strategies.
    
    Args:
        df: DataFrame with partition columns
        base_path: Directory to save partitioned data
    
    Returns:
        Dictionary with results for each strategy
    """
    # TODO: Implement this function
    # Strategy 1: No partitioning
    # Strategy 2: By year/month
    # Strategy 3: By year/month/day
    # Strategy 4: By category
    # For each: measure write time, count files, measure size
    pass


# Run partitioning benchmark
print("Creating partitioned datasets...")
print("(This may take a few minutes)\n")

partition_results = benchmark_partitioning(df_partitioned, PARTITION_DIR)

print("Results:")
print("-" * 60)
print(f"{'Strategy':<25} {'Files':<10} {'Size (MB)':<12} {'Write (s)':<12}")
print("-" * 60)
for name, data in partition_results.items():
    print(f"{name:<25} {data['num_files']:<10} {data['size_mb']:<12.1f} {data['write_sec']:<12.3f}")

### Part 4C: Benchmark Queries

In [ ]:
def benchmark_partition_queries(base_path: Path, year: int, month: int, 
                                 day: int, category: str) -> dict:
    """
    Benchmark query performance across partitioning strategies.
    
    Args:
        base_path: Base directory with partitioned data
        year, month, day: Date to filter for queries
        category: Category to filter for queries
    
    Returns:
        Dictionary with query times for each strategy
    """
    # TODO: Implement this function
    # Query 1: Specific day
    # Query 2: Category for one month
    # Query 3: Full aggregation
    # For each strategy, time each query
    pass


# Run query benchmarks
print("Benchmarking queries across partitioning strategies...\n")

query_results = benchmark_partition_queries(
    PARTITION_DIR,
    year=2024, month=1, day=15,
    category='Electronics'
)

print("Query Times (seconds):")
print("-" * 70)
print(f"{'Strategy':<25} {'Day Query':<15} {'Cat+Month':<15} {'Full Agg':<15}")
print("-" * 70)
for name, data in query_results.items():
    day_q = f"{data['query_day_sec']:.3f}" if data['query_day_sec'] else "N/A"
    cat_q = f"{data['query_category_month_sec']:.3f}" if data['query_category_month_sec'] else "N/A"
    full_q = f"{data['query_full_agg_sec']:.3f}" if data['query_full_agg_sec'] else "N/A"
    print(f"{name:<25} {day_q:<15} {cat_q:<15} {full_q:<15}")

### 💡 Key Insight: Partitioning Trade-offs

| Query Type | Best Strategy |
|------------|---------------|
| Specific day | Partition by year/month/day |
| Category analysis | Partition by category |
| Full aggregation | No partitioning (less overhead) |

**Rule**: Partition by columns you **filter** on frequently!

---

## 5. Reflection

**Your task:** Write a short reflection (3-5 sentences) answering:

1. What was the biggest memory reduction you achieved with dtype optimization?
2. Which storage format would you choose for a data warehouse? Why?
3. When would partitioning hurt rather than help performance?

In [ ]:
# TODO: Write your reflection here
reflection = """
Replace this text with your reflection.
Think about what you learned about data types and formats.
What will you do differently in your future projects?
""".strip()

print("Your reflection:")
print(reflection)

---

## 6. Save Results

In [ ]:
# Compile all results
results = {
    "lab": "03_data_types_formats",
    "timestamp": pd.Timestamp.now().isoformat(),
    "exercise_1_dtypes": {
        "baseline_memory_mb": baseline_memory['total_mb'],
        "optimized_memory_mb": optimized_memory['total_mb'],
        "reduction_factor": round(baseline_memory['total_mb'] / optimized_memory['total_mb'], 2),
        "groupby_speedup": round(groupby_baseline / groupby_optimized, 2),
        "filter_speedup": round(filter_baseline / filter_optimized, 2),
    },
    "exercise_2_formats": format_results,
    "exercise_3_parquet": {
        "row_group_sizes": rg_results,
        "predicate_pushdown": pushdown_results if 'pushdown_results' in dir() else None,
    },
    "exercise_4_partitioning": {
        "strategies": partition_results,
        "query_benchmarks": query_results,
    },
    "reflection": reflection,
}

# Save to JSON
with open(METRICS_PATH, "w") as f:
    json.dump(results, f, indent=2, default=str)

print(f"✓ Results saved to: {METRICS_PATH}")

---

## 🎉 Lab Complete!

### What You Learned

1. **Data Type Optimization**: Choosing the right dtype can reduce memory 5-10x
2. **Category dtype**: Essential for repeated strings, speeds up groupby operations
3. **Column-Oriented Storage**: Parquet reads only columns you need
4. **Compression Trade-offs**: Zstd offers the best balance of speed and ratio
5. **Partitioning**: Powerful for selective queries, but can hurt full scans

### Optimization Checklist

- ✅ Use smallest int type that fits your data
- ✅ Use `category` for strings with <50% unique values
- ✅ Use `float32` unless you need high precision
- ✅ Store data as Parquet with Zstd compression
- ✅ Partition by columns you filter on frequently

### Files to Submit

1. `notebooks/lab03_data_types_formats.ipynb` (this notebook)
2. `results/lab03_metrics.json`

---

**Next Lab**: We'll explore parallel processing and distributed computing!